<a href="https://colab.research.google.com/github/minhhieu9800/AIN501/blob/main/MSEK10_K11_NguyenMinhHieu_AIN501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Kết nối đến GitHub
!git clone https://github.com/minhhieu9800/AIN501.git

Cloning into 'AIN501'...
remote: Enumerating objects: 3540, done.
remote: Counting objects: 100% (270/270), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 3540 (delta 4), reused 200 (delta 3), pack-reused 3270
Receiving objects: 100% (3540/3540), 83.69 MiB | 11.95 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam
import keras.callbacks as callbacks
from keras.models import load_model

import requests
from PIL import Image
from io import BytesIO

In [4]:
# Khai báo đường dẫn cho tập dữ liệu train và validate
train_dir = '/content/AIN501/train'
val_dir = '/content/AIN501/validate'

# Thông số cho tập dữ liệu
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 32
epochs = 100
num_classes = len([f for f in os.listdir(train_dir) if not f.endswith('.zip')])

print(num_classes)

# Tạo data generator cho tập train và validate
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Xây dựng mô hình CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

# Lưu model
model.save('logo_model.h5')




45
Found 1228 images belonging to 45 classes.
Found 1669 images belonging to 45 classes.
Epoch 1/100
38/38 [==============================] - 23s 591ms/step - loss: 3.8976 - accuracy: 0.0293 - val_loss: 3.8079 - val_accuracy: 0.0373
Epoch 2/100
38/38 [==============================] - 22s 588ms/step - loss: 3.7628 - accuracy: 0.0485 - val_loss: 3.7713 - val_accuracy: 0.0721
Epoch 3/100
38/38 [==============================] - 22s 570ms/step - loss: 3.7024 - accuracy: 0.0602 - val_loss: 3.6923 - val_accuracy: 0.0950
Epoch 4/100
38/38 [==============================] - 21s 546ms/step - loss: 3.5994 - accuracy: 0.0987 - val_loss: 3.6032 - val_accuracy: 0.0980
Epoch 5/100
38/38 [==============================] - 21s 559ms/step - loss: 3.4605 - accuracy: 0.1438 - val_loss: 3.4773 - val_accuracy: 0.1599
Epoch 6/100
38/38 [==============================] - 22s 568ms/step - loss: 3.3305 - accuracy: 0.1714 - val_loss: 3.3161 - val_accuracy: 0.1905
Epoch 7/100
38/38 [============================

In [6]:
# # Đánh giá mô hình
score = model.evaluate(val_generator, workers=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

53/53 [==============================] - 5s 89ms/step - loss: 2.1059 - accuracy: 0.6872
Test loss: 2.1059486865997314
Test accuracy: 0.6872378587722778


In [12]:
def predict_logo(image_path):
  class_names = sorted(os.listdir(train_dir))
  model = load_model("/content/logo_model.h5")
  # đường dẫn đến file ảnh logo cần dự đoán
  # img_path = image_path

  url = image_path
  response = requests.get(url)

  # mở ảnh từ response
  img = Image.open(BytesIO(response.content))

  # Chuyển ảnh về RGB
  img = img.convert('RGB')

  img = img.resize((224, 224))

  img_array = np.array(img)
  x = np.expand_dims(img_array, axis=0)
  x = x / 255.0

  # dự đoán
  preds = model.predict(x)

  # print(class_names)
  # print(preds)
  # print(model.classes) 

  # lấy chỉ số của lớp có xác suất dự đoán cao nhất
  class_idx = np.argmax(preds[0])

  # print(class_idx)

  # in ra tên của hãng xe
  print(class_names[class_idx])


In [13]:
predict_logo("https://vinfastdongnai3s.com/wp-content/uploads/2021/10/logo-vinfast.png")

1/1 [==============================] - 0s 79ms/step
VINFAST


In [14]:
predict_logo("https://seeklogo.com/images/B/bmw-logo-248C3D90E6-seeklogo.com.png")

1/1 [==============================] - 0s 92ms/step
BMW


In [15]:
predict_logo("https://tse2.mm.bing.net/th?id=OIP.4FcSxuTj2K1VjJfzurYPxwHaEK&pid=Api&P=0")

1/1 [==============================] - 0s 85ms/step
VAUXHALL


In [16]:
# update new data from git

# thêm 1 hãng xe nữa: PAGANI

%cd AIN501/
!git pull

/content/AIN501
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 79 (delta 3), reused 79 (delta 3), pack-reused 0
Unpacking objects: 100% (79/79), 829.66 KiB | 13.17 MiB/s, done.
From https://github.com/minhhieu9800/AIN501
   f591f2b..d9ea2d2  main       -> origin/main
Updating f591f2b..d9ea2d2
Fast-forward
 train/PAGANI/PAGANI_0.jpeg     | Bin 0 -> 20999 bytes
 train/PAGANI/PAGANI_1.jpeg     | Bin 0 -> 19663 bytes
 train/PAGANI/PAGANI_10.jpeg    | Bin 0 -> 2128 bytes
 train/PAGANI/PAGANI_11.jpeg    | Bin 0 -> 2271 bytes
 train/PAGANI/PAGANI_12.jpeg    | Bin 0 -> 2626 bytes
 train/PAGANI/PAGANI_13.jpeg    | Bin 0 -> 2128 bytes
 train/PAGANI/PAGANI_14.jpeg    | Bin 0 -> 27962 bytes
 train/PAGANI/PAGANI_15.jpeg    | Bin 0 -> 22128 bytes
 train/PAGANI/PAGANI_16.jpeg    | Bin 0 -> 10003 bytes
 train/PAGANI/PAGANI_17.jpeg    | Bin 0 -> 19044 bytes
 train/PAGANI/PAGANI_18.jpeg    | Bin 0 -> 163

In [ ]:
# Lấy mô hình đã được huấn luyện trước đó
base_model = load_model("/content/logo_model.h5")

# Loại bỏ lớp cuối cùng của mô hình
base_model.layers.pop()

# Thêm một lớp mới tương ứng với số lượng logo mới
num_classes = len([f for f in os.listdir(train_dir) if not f.endswith('.zip')]) # số lượng logo mới

# Loại bỏ lớp cuối cùng của mô hình:
model.layers.pop()

# Đóng băng các lớp đã được huấn luyện trước đó
for layer in model.layers:
    layer.trainable = False

new_output = Dense(num_classes, activation='softmax')(base_model.layers[-1].output)

# Tạo mô hình mới
model = Model(inputs=base_model.inputs, outputs=new_output)

# Huấn luyện mô hình với tập dữ liệu mới, bao gồm cả logo cũ và logo mới
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

# Lưu lại mô hình đã được huấn luyện để sử dụng lại sau này
model.save('update_logo_model.h5')


Found 1257 images belonging to 46 classes.
Found 1716 images belonging to 46 classes.
Epoch 1/100
39/39 [==============================] - 24s 591ms/step - loss: 3.8164 - accuracy: 0.0743 - val_loss: 3.8058 - val_accuracy: 0.0531
Epoch 2/100
39/39 [==============================] - 22s 574ms/step - loss: 3.8020 - accuracy: 0.0735 - val_loss: 3.7952 - val_accuracy: 0.0584
Epoch 3/100
39/39 [==============================] - 22s 567ms/step - loss: 3.7928 - accuracy: 0.0898 - val_loss: 3.7829 - val_accuracy: 0.0637
Epoch 4/100
39/39 [==============================] - 23s 586ms/step - loss: 3.7835 - accuracy: 0.0931 - val_loss: 3.7766 - val_accuracy: 0.0649
Epoch 5/100
39/39 [==============================] - 21s 541ms/step - loss: 3.7737 - accuracy: 0.0955 - val_loss: 3.7654 - val_accuracy: 0.0702
Epoch 6/100
39/39 [==============================] - 22s 567ms/step - loss: 3.7696 - accuracy: 0.1045 - val_loss: 3.7663 - val_accuracy: 0.0731
Epoch 7/100
39/39 [==============================]